In [1]:
!pip install h5py

In [1]:
import h5py
h5f = h5py.File('./SVHN_single_grey1.h5', 'r')

# Load the training and the test set
X_train = h5f['X_train'][:]
y_train = h5f['y_train'][:]
X_test = h5f['X_test'][:]
y_test = h5f['y_test'][:]


# Close this file
h5f.close()

In [2]:
len(X_train), len(X_test),X_train.shape, X_test.shape

(42000, 18000, (42000, 32, 32), (18000, 32, 32))

In [3]:
X_train = X_train.reshape(X_train.shape[0], 1024)
X_test = X_test.reshape(X_test.shape[0], 1024)
# Normalize inputs from 0-255 to 0-1

X_train = X_train/255
X_test = X_test/255

In [4]:
svh_x_train, svh_x_test, svh_y_train, svh_y_test = X_train, X_test, y_train, y_test

In [5]:
from util import Utils

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
from sklearn.model_selection import train_test_split

svh_X_train, svh_X_val, svh_Y_train, svh_y_val = train_test_split(svh_x_train, svh_y_train, test_size=1/3, random_state=42)

In [7]:
svh_y_train

array([2, 6, 7, ..., 7, 0, 4], dtype=uint8)

In [8]:
best_svh_model = Utils.find_best_rf_parameters(svh_x_train, svh_y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits

All Results:
{'n_estimators': 100, 'max_features': 'log2', 'max_depth': 10, 'criterion': 'gini'} with accuracy: 0.6210238095238095
{'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'entropy'} with accuracy: 0.6234761904761905
{'n_estimators': 10, 'max_features': 'sqrt', 'max_depth': 100, 'criterion': 'entropy'} with accuracy: 0.5524285714285715
{'n_estimators': 10, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'gini'} with accuracy: 0.5052619047619048
{'n_estimators': 50, 'max_features': 'log2', 'max_depth': 50, 'criterion': 'gini'} with accuracy: 0.6424761904761905
{'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 100, 'criterion': 'gini'} with accuracy: 0.6792857142857143
{'n_estimators': 100, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'gini'} with accuracy: 0.6290238095238095
{'n_estimators': 10, 'max_features': 'sqrt', 'max_depth': 50, 'criterion': 'entropy'} with accur

In [10]:
Utils.accuracy_measure_rf(svh_x_train, svh_y_train, 
                          svh_x_test, svh_y_test, 
                          n_estimators = 50, criterion = 'entropy', max_depth = 50, max_features = 'sqrt')

--- 364.29574751853943 seconds ---


(0.7139444444444445,
 (array([0.67918089, 0.67537827, 0.75722543, 0.64940912, 0.74713243,
         0.69662921, 0.69589977, 0.80742459, 0.70892626, 0.73800738]),
  array([0.76791621, 0.80579869, 0.72656683, 0.67132054, 0.79083885,
         0.66628959, 0.66703057, 0.7699115 , 0.60485651, 0.66518847]),
  array([0.72082794, 0.7348466 , 0.74157939, 0.66018307, 0.76836461,
         0.68112171, 0.68115942, 0.78822197, 0.65276951, 0.69970845]),
  array([1814, 1828, 1803, 1719, 1812, 1768, 1832, 1808, 1812, 1804])))

In [ ]:
best_xgb_model = Utils.find_best_xgb_parameters(svh_X_train, svh_Y_train, svh_X_val, svh_y_val)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
